In [550]:
import sympy as sy
import numpy as np
import math

from enum import Enum
from typing import List


In [551]:
class Signs(Enum):
    LESS = 1
    EQUAL = 2
    GREAT = 3


class TaskType(Enum):
    MIN = 1
    MAX = 2


class Inequality:
    def __init__(self, coeff: List[int], sign: Signs, free: int):
        self.coeff = coeff
        self.sign = sign
        self.free = free


class Task:
    def __init__(self, main_coeff: List[int], task_type: TaskType, inequalities: List[Inequality]):
        self.main_coeff = main_coeff
        self.task_type = task_type
        self.inequalities = inequalities


In [552]:
def check(actual, expected):
    if isinstance(actual, list):
        ok = True
        for i in range(len(actual)):
            if actual[i] != expected[i]:
                ok = False
        if not ok:
            print('points are different')
    else:
        if actual != expected:
            print(f'not equal. expected: {expected}, got: {actual}')


In [553]:
def to_rational_array(a):
    sa = a.shape
    return np.array([sy.Rational(x) for x in a.reshape(-1)]).reshape(sa)


In [554]:
def simplex1(matrix_a, column_b, functional, initial_basis):
    functional = np.append(functional, 0)

    matrix_a = to_rational_array(matrix_a)
    column_b = to_rational_array(column_b)
    functional = to_rational_array(functional)

    data = np.c_[matrix_a, column_b * -1]

    n = matrix_a.shape[1]
    m = matrix_a.shape[0]

    # print(data)
    # print(functional)
    # print(data.shape)
    # print(initial_basis)

    # to ones on basis elements
    m_to_basis = [-1] * m
    for j in range(m):
        cnt = j
        ind = initial_basis[j]
        if data[cnt][ind] == 0:
            # если в
            for i in range(cnt + 1, m):
                if data[i][ind] != 0:
                    data[cnt] += data[i]
                    break
        data[cnt] /= data[cnt][ind]

        for i in range(m):
            if i == cnt:
                continue
            data[i] -= data[cnt] * data[i][ind]
        m_to_basis[cnt] = ind


    for i in range(m):
        j = m_to_basis[i]
        functional -= data[i] * (functional[j] / data[i][j])

    # print("after gauss")
    # print(data)
    # print(functional)
    # print("----------")

    for cyc in range(100):
        ind_in_basis, ind_out_basis = -1, -1
        for i in range(n):
            if functional[i] > 0:
                ind_in_basis = i
                break
        if ind_in_basis == -1:
            break
        prev_rez = -math.inf
        for j in range(m):
            if data[j][ind_in_basis] > 0:
                delta = data[j][-1] / data[j][ind_in_basis]
                if 0 >= delta > prev_rez:
                    ind_out_basis = j
                    prev_rez = delta
        if ind_out_basis == -1:
            print('unlimited')
            break

        m_to_basis[ind_out_basis] = ind_in_basis

        data[ind_out_basis] /= data[ind_out_basis][ind_in_basis]
        for i in range(m):
            if i != ind_out_basis:
                data[i] -= data[ind_out_basis] * data[i][ind_in_basis]

        functional -= data[ind_out_basis] * functional[ind_in_basis]

        # print(cyc)
        # print(prev_rez)
        # print(ind_in_basis, ind_out_basis)
        # print(data)
        # print(functional)
        # print("----------")

    opt_point = [0] * n
    for i in range(m):
        opt_point[m_to_basis[i]] = -data[i][-1]

    mbs = sorted(m_to_basis)

    return -functional[-1], opt_point, mbs


In [555]:
def solve(task: Task, initial_basis: List[int] = None):
    # transform data from dsl to np.array
    matrix_a = np.array(
        [inequality.coeff for inequality in task.inequalities],
        dtype=np.float64
    )
    column_b = np.array(
        [inequality.free for inequality in task.inequalities],
        dtype=np.float64
    )
    main_functional = np.array(task.main_coeff, dtype=np.float64)
    if task.task_type == TaskType.MIN:
        main_functional *= -1

    m = matrix_a.shape[0]
    n = matrix_a.shape[1]

    for i in range(m):
        inequality = task.inequalities[i]
        if inequality.sign == Signs.GREAT:
            matrix_a[i] *= -1
            column_b[i] *= -1
            # TODO check: sign changed from > to <
        if inequality.sign != Signs.EQUAL:
            addition_column = np.zeros(m)
            addition_column[i] = 1
            matrix_a = np.c_[
                matrix_a,
                addition_column
            ]
            main_functional = np.append(main_functional, 0)

    n2 = matrix_a.shape[1]

    # выполняем метод искуственного базиса, если начальный базис не передан явно
    if initial_basis is None:
        matrix_y = np.eye(m)
        for i in range(m):
            if column_b[i] < 0:
                matrix_y[i][i] = -1
        a_matrix_ay = np.c_[matrix_a, matrix_y]
        a_functional = np.array([0] * n2 + [-1] * m)
        a_in_func = list(range(n2, n2 + m))
        # print("ib0:", a_in_func)
        opt_value, point, mbs = simplex1(a_matrix_ay, column_b, a_functional, a_in_func)

        # print(">", opt_value, point)
        if abs(opt_value) != 0:
            print("not feasible")
            return 0, []

        initial_basis = mbs
        # print('ib: ', initial_basis)

    opt_value, opt_point, _mbs2 = simplex1(matrix_a, column_b, main_functional, initial_basis)

    opt_value *= 1 if task.task_type == TaskType.MIN else -1
    opt_point = opt_point[:n]

    print("optimal value", opt_value)
    print("in point:", opt_point)

    return opt_value, opt_point


In [556]:
task_1 = Task(
    [-6, -1, -4, 5], TaskType.MIN,
    [
        Inequality([3, 1, -1, 1], Signs.EQUAL, 4),
        Inequality([5, 1, 1, -1], Signs.EQUAL, 4)
    ]
)

result_1_value, result_1_point = solve(
    task_1,
    [0, 3]
)

check(result_1_value, -4)
check(result_1_point, [0, 4, 0, 0])

optimal value -4
in point: [0, 4, 0, 0]


In [557]:
result_1_2_value, result_1_2_point = solve(task_1)

check(result_1_2_value, -4)
check(result_1_2_point, [0, 4, 0, 0])


optimal value -4
in point: [0, 4, 0, 0]


In [558]:
#TODO дробные числа

task_2 = Task(
    [-1, -2, -3, 1], TaskType.MIN,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2_value, result_2_point = solve(
    task_2,
    [1, 2]
)

check(result_2_value, -6)
check(result_2_point, [2, 2, 0, 0])


optimal value -6
in point: [2, 2, 0, 0]


In [559]:
task_2_max = Task(
    [-1, -2, -3, 1], TaskType.MAX,
    [
        Inequality([1, -3, -1, -2], Signs.EQUAL, -4),
        Inequality([1, -1, 1, 0], Signs.EQUAL, 0)
    ]
)

result_2_max_value, result_2_max_point = solve(task_2_max)

check(result_2_max_value, 2)
check(result_2_max_point, [0, 0, 0, 2])


optimal value 2
in point: [0, 0, 0, 2]


In [560]:
task_3 = Task(
    [-1, -2, -1, 3, -1], TaskType.MIN,
    [
        Inequality([1, 1, 0, 2, 1], Signs.EQUAL, 5),
        Inequality([1, 1, 1, 3, 2], Signs.EQUAL, 9),
        Inequality([0, 1, 1, 2, 1], Signs.EQUAL, 6)
    ]
)

result_3_value, result_3_point = solve(task_3)

check(result_3_value, -11)
check(result_3_point, [3, 2, 4, 0, 0])


optimal value -11
in point: [3, 2, 4, 0, 0]


In [561]:
task4 = Task(
    [-1, -1, -1, 1, -1], TaskType.MIN,
    [
        Inequality([1, 1, 2, 0, 0], Signs.EQUAL, 4),
        Inequality([0, -2, -2, 1, -1], Signs.EQUAL, -6),
        Inequality([1, -1, 6, 1, 1], Signs.EQUAL, 12)
    ]
)

result_4_value, result_4_point = solve(task4)

check(result_4_value, -10)
check(result_4_point, [4, 0, 0, 6, 12])


optimal value -10
in point: [4, 0, 0, 1, 7]
points are different


In [562]:
#TODO побороть зацикливание

task5 = Task(
    [-1, 4, -3, 10], TaskType.MIN,
    [
        Inequality([1, 1, -1, -10], Signs.EQUAL, 0),
        Inequality([1, 14, 10, -10], Signs.EQUAL, 11),
    ]
)

result_5_value, result_5_point = solve(
    task5,
    # [1, 3]
)

check(result_5_value, -4)
check(result_5_point, [1, 0, 1, 0])


optimal value -4
in point: [1, 0, 1, 0]


In [563]:
task_6 = Task(
    [-1, 5, 1, -1], TaskType.MIN,
    [
        Inequality([1, 3, 3, 1], Signs.LESS, 3),
        Inequality([0, 2, 3, -1], Signs.LESS, 4),
    ]
)

result_6_value, result_6_point = solve(task_6)

check(result_6_value, -3)
check(result_6_point, [3, 0, 0, 0])


optimal value -3
in point: [3, 0, 0, 0]


In [564]:
task_7 = Task(
    [-1, -1, 1, -1, 2], TaskType.MIN,
    [
        Inequality([3, 1, 1, 1, -2], Signs.EQUAL, 10),
        Inequality([6, 1, 2, 3, -4], Signs.EQUAL, 20),
        Inequality([10, 1, 3, 6, -7], Signs.EQUAL, 30)
    ]
)

result_7_value, result_7_point = solve(task_7)

check(result_7_value, 10)
check(result_7_point, [0, 0, 10, 0, 0])


optimal value 10
in point: [0, 0, 10, 0, 0]
